# Week 2

In [478]:
import pandas as pd
df=pd.read_csv('stocks/amzn.us.txt')

In [479]:
split_index=int(0.8*len(df))

In [480]:
df['Daily Return']=df['Close'].pct_change()*100
df['7-day Moving Average']=df['Close'].rolling(window=7).mean()
df['30-day Moving Average']=df['Close'].rolling(window=30).mean()
df['Rolling Volatility (30d)']=df['Daily Return'].rolling(window=30).std()
df['Lag_Close_1'] = df['Close'].shift(1)
df['Lag_Close_2'] = df['Close'].shift(2)
df['Lag_Return_1'] = df['Daily Return'].shift(1)
df=df.dropna()

In [481]:
features=['Open', 'OpenInt',
        '7-day Moving Average', '30-day Moving Average',
       'Rolling Volatility (30d)','Lag_Close_1','Lag_Close_2','Lag_Return_1']
X = df[features]
y = df['Close']

# Split into train/test (80/20)
split_index = int(len(df) * 0.8)
X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

from sklearn import linear_model
linear_reg=linear_model.LinearRegression()
linear_reg.fit(X_train,y_train)
# print(linear_reg.coef_)
# print(linear_reg.intercept_)


LinearRegression()

In [482]:
y_pred=linear_reg.predict(X_test)

/Users/aradhyaagrawal/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning:

divide by zero encountered in matmul

/Users/aradhyaagrawal/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning:

overflow encountered in matmul

/Users/aradhyaagrawal/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning:

invalid value encountered in matmul



In [483]:
from sklearn.metrics import mean_absolute_error
linear_reg_mae=mean_absolute_error(y_test,y_pred)
print(f'MAE: {linear_reg_mae:.4f}')
# print(df['Close'].mean())

MAE: 5.4110


In [484]:
# import plotly.io as pio
# import plotly.graph_objects as go
# import numpy as np
# pio.renderers.default = 'browser'

# # Create x-axis index
# x = np.arange(len(y_test))

# # Create the plot
# fig = go.Figure()

# # Add actual values trace
# fig.add_trace(go.Scatter(
#     x=x, y=y_test, mode='lines', name='Actual', line=dict(color='blue')
# ))

# # Add predicted values trace
# fig.add_trace(go.Scatter(
#     x=x, y=y_pred, mode='lines', name='Predicted', line=dict(color='red', dash='dash')
# ))

# # Customize layout
# fig.update_layout(
#     title='Actual vs Predicted Prices',
#     xaxis_title='Date',
#     yaxis_title='Close',
#     legend=dict(x=0, y=1),
#     template='plotly_white',
#     width=900,
#     height=500
# )

# # Show plot
# fig.show()


The mean absolute error from linear regression is `MAE: 5.4110`, which is decent as compared to the last closing prices which are around 1100 in 2017.

In [485]:
from statsmodels.tsa.arima.model import ARIMA
arima_model = ARIMA(y_train, order=(6, 2, 8))  # (p,d,q) --> You may tune these
arima_model_fit = arima_model.fit()

/Users/aradhyaagrawal/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.

/Users/aradhyaagrawal/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.

/Users/aradhyaagrawal/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.

/Users/aradhyaagrawal/Library/Python/3.9/lib/python/site-packages/numpy/linalg/_linalg.py:3220: RuntimeWarning:

divide by zero encountered in matmul

/Users/aradhyaagra

In [486]:
arima_forecast=arima_model_fit.forecast(steps=len(y_test))  

/Users/aradhyaagrawal/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

/Users/aradhyaagrawal/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.



In [487]:
arima_mae=mean_absolute_error(y_test,arima_forecast)
print(f"MAE: {arima_mae:0.4f}")

MAE: 205.4185


The mean absolute error obtained from ARIMA model is `MAE: 205.4185`.

## Random Forests

In [488]:
from sklearn.ensemble import RandomForestRegressor

In [489]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [490]:
y_pred = rf_model.predict(X_test)

In [491]:
rf_mae = mean_absolute_error(y_test, y_pred)
print(f"Random Forest MAE: {rf_mae:.4f}")

Random Forest MAE: 275.0564


In [ ]:
# import plotly.graph_objects as go
# fig = go.Figure()

# fig.add_trace(go.Scatter(y=y_test.values, mode='lines', name='Actual'))
# fig.add_trace(go.Scatter(y=y_pred, mode='lines', name='Predicted', line=dict(dash='dash')))

# fig.update_layout(title='Random Forest Prediction vs Actual',
#                   xaxis_title='Time Index',
#                   yaxis_title='Price',
#                   template='plotly_white')

# fig.show()

In [493]:
print(y_train.describe())
print(y_test.describe())

count    4098.000000
mean       81.118824
std        76.793555
min         1.520000
25%        30.902500
50%        49.515000
75%        95.022500
max       320.950000
Name: Close, dtype: float64
count    1025.000000
mean      589.449956
std       239.282959
min       286.950000
25%       362.700000
50%       550.190000
75%       776.320000
max      1132.880000
Name: Close, dtype: float64


The problem lies in the split of the data training and testing, the average of training data is 81 while that of the testing data is 590.
Random Forest cannot extrapolate.
It can only predict within the range of values it has seen in the training set. <br> Hence such a high error.

## Additional Experiments

* Tuning the data split to 0.9 train and 0.1 test significantly improves the Random Forest Results.
* I have experimented from many possible values for p,q and d for the ARIMA model and finally chose which gave the best MAE.
* To improve model accuracy, various lag features were added to have model memory.
* Added plotly curves at required locations for better visualization of comparision between actual and predicted y.